<a href="https://colab.research.google.com/github/amirda9/TelegramBot/blob/master/bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#  !pip install python-telegram-bot --upgrade
# !pip install python-telegram-bot-calendar


In [ ]:
import logging
from telegram import Update,ReplyKeyboardMarkup, ReplyKeyboardRemove,InlineKeyboardMarkup,InlineKeyboardButton
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext, ConversationHandler, \
    CallbackQueryHandler
from telegram_bot_calendar import DetailedTelegramCalendar, LSTEP
from datetime import date

# your_translation_months = list('abcdefghijkl')
# your_translation_days_of_week = list('ش ی د س چ پ ج')


class MyStyleCalendar(DetailedTelegramCalendar):
    # previous and next buttons style. they are emoji now!
    prev_button = "⬅️"
    next_button = "➡️"
    start = date(1360,12,15)
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.days_of_week['en'] = ['','','','','','','']
        self.months['en'] = ['فروردین','اردیبهشت','خرداد','تیر','مرداد','شهریور','مهر','آبان','آذز','دی','بهمن','اسفند']
        # self.max_date = date.today()
        self.size_year = 4
        # self.max_date = date(1399, 3, 3)
        self.middle_bytton_year = 1370
        

class BOT:
    def __init__(self, token,name):
        self.token = token
        self.name = name
        # Enable logging
        logging.basicConfig(
            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
        )
        self.logger = logging.getLogger(__name__)
        self.updater = Updater(token)

        # # Get the dispatcher to register handlers
        self.dispatcher = self.updater.dispatcher
        #
        # # on different commands - answer in Telegram
        # self.dispatcher.add_handler(CommandHandler("start", self.start))
        # self.dispatcher.add_handler(CommandHandler("help", self.help_command))

        conv_handler = ConversationHandler(
            entry_points=[CommandHandler('start', self.start),CommandHandler('amir', self.amir)],
            states={
                0: [MessageHandler(Filters.text, self.first_name_get)],
                11:[CallbackQueryHandler(self.button)],
                1: [MessageHandler(Filters.text, self.last_name_get)],
                2: [MessageHandler(Filters.text, self.phone_get)],
                3: [MessageHandler(Filters.text, self.mail_get)],
                4: [CallbackQueryHandler(self.cal)],
                5 :[CallbackQueryHandler(self.btn_level)]
            },
            fallbacks=[CommandHandler('cancel', self.start)],


        )

        self.dispatcher.add_handler(conv_handler)

        # self.dispatcher.add_handler(CallbackQueryHandler(self.cal))
        # updater.dispatcher.add_handler(CallbackQueryHandler(DetailedTelegramCalendar.func))
        # on noncommand i.e message - echo the message on Telegram
        # dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, self.echo))

        # Start the Bot
        self.updater.start_polling()

        # Run the bot until you press Ctrl-C or the process receives SIGINT,
        # SIGTERM or SIGABRT. This should be used most of the time, since
        # start_polling() is non-blocking and will stop the bot gracefully.
        self.updater.idle()



    def amir(self, update: Update, context: CallbackContext) -> int:
        calendar, step = MyStyleCalendar().build()
        update.message.reply_text(
            f"Select {LSTEP[step]}",
            reply_markup=calendar)
        # update.message.reply_text("Please select a date: ",
        #                           reply_markup=telegramcalendar.create_calendar())
        return 4
    def start(self, update: Update, context: CallbackContext) -> int:
        """Send a message when the command /start is issued."""
        keyboard = [
            [InlineKeyboardButton("شروع ثبت نام", callback_data='1')],
        ]

        reply_markup = InlineKeyboardMarkup(keyboard)
        update.message.reply_text(str(self.name))
        update.message.reply_text("خوش آمدید", reply_markup=reply_markup)


        return 11


    def help(self, update: Update, context: CallbackContext) -> int:
        return 12

    def button(self,update: Update, context: CallbackContext) -> int:
        query = update.callback_query
        query.answer()

        query.edit_message_text(text=f"لطفا نام خود را وارد کنید")

        return 0



    def btn_level(self,update: Update, context: CallbackContext) -> int:
        query = update.callback_query
        self.level = query.data

        query.edit_message_text(text=f"ثبت نام با موفقیت انجام شد")

        return 6



    def help_command(self, update: Update, context: CallbackContext) -> None:
        """Send a message when the command /help is issued."""
        update.message.reply_text('Help!')

    def first_name_get(self,update: Update, context: CallbackContext) -> None:
        self.first_name = update.message.text
        update.message.reply_text("لطفا نام خانوادگی خود را وارد کنید")
        # self.dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, self.last_name_get))
        self.dispatcher.add_handler(CommandHandler("start", self.start))
        return 1
    def last_name_get(self,update: Update, context: CallbackContext) -> None:
        self.last_name = update.message.text
        # update.message.reply_text("YOUR LAST NAME is : \t"+str(update.message.text) + "\nYour Full Name is :" +str(self.first_name) +" " +self.last_name)
        update.message.reply_text(str(self.first_name) + " " + str(self.last_name) + " عزیز " + "\n لطفا شماره تلفن خود را وارد کنید")
        return 2

    def phone_get(self,update: Update, context: CallbackContext) -> None:
        self.phone_number = update.message.text
        # update.message.reply_text("YOUR Phone NO is : \t"+str(update.message.text))
        update.message.reply_text("لطفا ایمیل خود را وارد کنید")
        return 3

    def mail_get(self,update: Update, context: CallbackContext) -> None:
        self.mail = update.message.text
        # update.message.reply_text(str(self.first_name) +" " +self.last_name +"\n" + self.phone_number +  "\n"+str(update.message.text))
        calendar, step = MyStyleCalendar().build()
        update.message.reply_text(f"Select {LSTEP[step]}",
                                  reply_markup=calendar)
        return 4

    def cal(self, update: Update, context: CallbackContext) -> None:
        # selected, date = telegramcalendar.process_calendar_selection(self.dispatcher.bot, update)
        # if selected:
        #     self.dispatcher.bot.send_message(chat_id=update.callback_query.from_user.id,
        #                      text="You selected %s" % (date.strftime("%d/%m/%Y")),
        #                      reply_markup=ReplyKeyboardRemove())
        result, key, step = MyStyleCalendar().process(update.callback_query.data)
        if not result and key:
            self.dispatcher.bot.edit_message_text(f"لطفا تاریخ تولد خود را انتخاب کنید",
                                  update.callback_query.message.chat.id,
                                  update.callback_query.message.message_id,
                                  reply_markup=key)
        elif result:
            self.birth = result
            keyboard = [
            [InlineKeyboardButton("مبتدی", callback_data='1')],
            [InlineKeyboardButton("متوسط", callback_data='2')],
            [InlineKeyboardButton("حرفه ای", callback_data='3')]
            ]
            reply_markup = InlineKeyboardMarkup(keyboard)
            self.dispatcher.bot.edit_message_text(f" در چه سطحی از ترید هستید ؟",
                                  update.callback_query.message.chat.id,
                                  update.callback_query.message.message_id,reply_markup=reply_markup)
            # self.level_get(update,context=CallbackContext)
            return 5

    # def level_get(self,update: Update, context: CallbackContext) -> int:
        
    #     keyboard = [
    #         [InlineKeyboardButton("مبتدی", callback_data='1')],
    #         [InlineKeyboardButton("متوسط", callback_data='2')],
    #         [InlineKeyboardButton("حرفه ای", callback_data='3')]
    #         ]
    #     reply_markup = InlineKeyboardMarkup(keyboard)
    #     update.message.reply_text(f"سطح ؟",reply_markup=reply_markup)
        # user = update.message.from_user
        # self.logger.info("Gender of %s: %s", user.first_name, update.message.text)
        # update.message.reply_text(
        #     'I see! Please send me a photo of yourself, '
        #     'so I know what you look like, or send /skip if you don\'t want to.',
        #     reply_markup=ReplyKeyboardRemove(),
        # )
        # self.level = update.message.text
        # update.message.reply_text(str(self.first_name) +" " +self.last_name +"\n" + self.phone_number +  "\n"+str(update.message.text))
        # return 4

if __name__ == '__main__':
    my_bot = BOT("1627766916:AAEcIdySaRXKyOftSAXsquN2BlzCOXtU8K8","Amir Bayat")

/usr/local/lib/python3.7/dist-packages/telegram/ext/conversationhandler.py:280: UserWarning: If 'per_message=False', 'CallbackQueryHandler' will not be tracked for every message.
  "If 'per_message=False', 'CallbackQueryHandler' will not be "
2021-03-03 11:57:48,694 - apscheduler.scheduler - INFO - Scheduler started
